In [ ]:
import os
import cv2
from ultralytics import YOLO
import mediapipe as mp
import time
import numpy as np

# === 設定相對路徑 ===
YOLO_DISPENSER_MODEL = os.path.join("dispensor weight", "best(yolov11).pt")
YOLO_PEOPLE_MODEL = os.path.join("dispensor weight", "yolo11n.pt")
VIDEO_INPUT = os.path.join("dispensor weight", "test2.mp4")
VIDEO_OUTPUT = os.path.join("Output Video", "processed_video.mp4")

# === Mediapipe 初始化 ===
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.3, min_tracking_confidence=0.3)
mp_draw = mp.solutions.drawing_utils

# === 載入 YOLO TensorRT 模型 ===
model_people = YOLO(YOLO_PEOPLE_MODEL)
model_dispenser = YOLO(YOLO_DISPENSER_MODEL)

# === 全域參數 ===
dispenser_roi = None
sanitized_count = 0
sanitized_ids = set()
track_state = {}
DELAY_TIME = 3
DISPENSER_DETECT_FRAMES = 100
dispenser_detected = False

# === 初始化影片 ===
def initialize_video(input_path, output_path):
    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))
    return cap, out, fps, frame_width, frame_height

# === 偵測酒精機位置 + 畫框 ===
def detect_dispenser(frame):
    results = model_dispenser(frame)
    for r in results:
        for box in r.boxes:
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            confidence = box.conf[0]
            if label.lower() == "dispenser" and confidence > 0.3:
                coords = list(map(int, box.xyxy[0]))
                cv2.rectangle(frame, (coords[0], coords[1]), (coords[2], coords[3]), (255, 0, 255), 2)
                cv2.putText(frame, "Dispenser", (coords[0], coords[1] - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 0, 255), 2)
                print(f"\n✅ Dispenser detected at: {coords}")
                return coords
    return None

# === 主人物追蹤與手部對應邏輯 ===
def process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count):
    global sanitized_count

    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb)

    height = dispenser_roi[3] - dispenser_roi[1]
    lower_y1 = dispenser_roi[1] + int(height * 0.8)
    lower_y2 = dispenser_roi[3] + int(height * 0.3)
    overlay = frame.copy()
    cv2.rectangle(overlay, (dispenser_roi[0], lower_y1), (dispenser_roi[2], lower_y2), (0, 255, 255), -1)
    frame[:] = cv2.addWeighted(overlay, 0.3, frame, 0.7, 0)

    triggered_hand = None
    if results.multi_hand_landmarks:
        print(f"🖐️ Detected {len(results.multi_hand_landmarks)} hand(s)")
        for i, hand_landmarks in enumerate(results.multi_hand_landmarks):
            wrist = hand_landmarks.landmark[mp_hands.HandLandmark.WRIST]
            index_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_MCP]
            pinky_mcp = hand_landmarks.landmark[mp_hands.HandLandmark.PINKY_MCP]

            wx, wy = int(wrist.x * w), int(wrist.y * h)
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.circle(frame, (wx, wy), 6, (0, 0, 255), -1)

            v1 = np.array([index_mcp.x - wrist.x, index_mcp.y - wrist.y, index_mcp.z - wrist.z])
            v2 = np.array([pinky_mcp.x - wrist.x, pinky_mcp.y - wrist.y, pinky_mcp.z - wrist.z])
            palm_normal = np.cross(v1, v2)
            palm_normal_unit = palm_normal / np.linalg.norm(palm_normal)
            dot = np.dot(palm_normal_unit, np.array([0, 0, -1]))

            if dot > 0.3 and dispenser_roi[0] <= wx <= dispenser_roi[2] and lower_y1 <= wy <= lower_y2:
                triggered_hand = (wx, wy)
                print(f"✅ Hand {i} entered zone with upward palm (dot={dot:.2f})")
                break

    people_boxes = {}
    for r in results_people:
        if r.boxes.id is None:
            continue
        for box, track_id_tensor in zip(r.boxes, r.boxes.id):
            track_id = int(track_id_tensor.item())
            cls_id = int(box.cls.item())
            label = r.names[cls_id]
            if label != "person":
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            people_boxes[track_id] = (x1, y1, x2, y2)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"ID: {track_id}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    if triggered_hand and people_boxes:
        min_dist = float('inf')
        target_id = None
        for track_id, (x1, y1, x2, y2) in people_boxes.items():
            cx = (x1 + x2) // 2
            cy = (y1 + y2) // 2
            dist = ((cx - triggered_hand[0]) ** 2 + (cy - triggered_hand[1]) ** 2) ** 0.5
            if dist < min_dist:
                min_dist = dist
                target_id = track_id
        print(f"🎯 Closest person to hand: ID {target_id} (distance={min_dist:.2f})")

        if target_id is not None:
            track_state.setdefault(target_id, {'last_detected': 0, 'show_text': False})
            if (current_time - track_state[target_id]['last_detected']) > DELAY_TIME:
                if target_id not in sanitized_ids:
                    sanitized_ids.add(target_id)
                    sanitized_count += 1
                    print(f"✅ Person ID {target_id} triggered disinfection")
                track_state[target_id] = {'last_detected': current_time, 'show_text': True}

            if track_state[target_id]['show_text']:
                cv2.putText(frame, f"detected {target_id} Sanitization Success!", (w // 4, h // 2),
                            cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

# === 主程式 ===
cap, out, fps, frame_width, frame_height = initialize_video(VIDEO_INPUT, VIDEO_OUTPUT)
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        print("處理完成")
        break

    current_time = time.time()
    h, w, _ = frame.shape

    if not dispenser_detected and frame_count < DISPENSER_DETECT_FRAMES:
        roi_candidate = detect_dispenser(frame)
        if roi_candidate:
            dispenser_roi = roi_candidate
            dispenser_detected = True
            print("Dispenser position fixed:", dispenser_roi)

    if dispenser_roi:
        results_people = model_people.track(frame, persist=True, tracker="botsort.yaml")
        process_tracking_optimized(frame, results_people, dispenser_roi, w, h, current_time, frame_count)

    cv2.putText(frame, f"Sanitized Count: {sanitized_count}", (10, 50),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    out.write(frame)

    display_frame = cv2.resize(frame, (1280, 720))
    cv2.imshow("Jetson Nano Sanitization Detection", display_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
    frame_count += 1

cap.release()
out.release()
cv2.destroyAllWindows()

WARNING ⚠️ Python>=3.10 is required, but Python==3.9.19 is currently installed 



INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


0: 640x384 (no detections), 44.8ms
Speed: 3.2ms preprocess, 44.8ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 67.5ms
Speed: 3.5ms preprocess, 67.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 51.2ms
Speed: 1.8ms preprocess, 51.2ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 50.2ms
Speed: 2.7ms preprocess, 50.2ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 45.9ms
Speed: 1.7ms preprocess, 45.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 51.9ms
Speed: 1.8ms preprocess, 51.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 40.0ms
Speed: 1.8ms preprocess, 40.0ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 384)

0: 640x384 (no detections), 37.6ms
Speed: 1.7ms preprocess, 37.6ms in

/opt/homebrew/lib/python3.9/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


🖐️ Detected 1 hand(s)

0: 640x384 1 parking meter, 43.5ms
Speed: 1.7ms preprocess, 43.5ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 1 hand(s)

0: 640x384 1 parking meter, 46.5ms
Speed: 2.1ms preprocess, 46.5ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 1 hand(s)

0: 640x384 1 parking meter, 47.2ms
Speed: 1.8ms preprocess, 47.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 1 hand(s)

0: 640x384 1 parking meter, 39.0ms
Speed: 1.6ms preprocess, 39.0ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 1 hand(s)

0: 640x384 1 person, 1 parking meter, 40.9ms
Speed: 1.7ms preprocess, 40.9ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 1 hand(s)

0: 640x384 1 person, 1 parking meter, 36.2ms
Speed: 1.7ms preprocess, 36.2ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 384)
🖐️ Detected 2 hand(s)

0: 640x384 1 person, 1 park